# SERLI - AI4Industry - 2025

---
Notebook du Groupe rouge foncé (on aurait préféré le bleu)
---

## Import des librairies

In [23]:
import cv2
import os
import json
from tqdm import tqdm
import numpy as np
import threading

## Variables globales

In [24]:
DATASET_FOLDER = "dataset/"
IMAGE_SIZE = (848, 480)
MASK_FILE = "mask1.png"

## Variables système

## Import des données

In [25]:
data_lock = threading.Lock() 
image_list = []
label_list = []

mask = cv2.imread(MASK_FILE)

def read_subfolder(subfolder_path, image_size, image_list, label_list):
    print(f"Reading subfolder: {subfolder_path}")
    for file in os.listdir(subfolder_path):
        if file.endswith('.png'):
            image_path = os.path.join(subfolder_path, file)
            file_name = file.split(".")[0]
            json_path = os.path.join(subfolder_path, file_name + ".json")

            index = int(file_name.split("_")[1])

            if os.path.exists(json_path) and os.path.exists(image_path):
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)
                image = cv2.bitwise_and(image, mask)

                with data_lock:
                    image_list[index] = image
                    with open(json_path) as json_file:
                        data = json.load(json_file)
                        label_list[index] = data
            else:
                print(f"json file does not exist for: {file}")
    print(f"Finished reading subfolder: {subfolder_path}")

if os.path.exists(DATASET_FOLDER):
    print("Dataset folder exists")
    
    dataset_len = 1
    for subfolders in os.listdir(DATASET_FOLDER):
        subfolder_path = os.path.join(DATASET_FOLDER, subfolders)
        for file in os.listdir(subfolder_path):
            if file.endswith('.png'):
                dataset_len += 1
                
    image_list = np.zeros((dataset_len, IMAGE_SIZE[1], IMAGE_SIZE[0], 3), dtype=np.uint8)
    label_list = np.zeros((dataset_len, 1), dtype=object)
    
    threads = []

    # Lancer un thread pour chaque sous-dossier
    for subfolder in os.listdir(DATASET_FOLDER):
        subfolder_path = os.path.join(DATASET_FOLDER, subfolder)
        if os.path.isdir(subfolder_path):
            thread = threading.Thread(target=read_subfolder, args=(subfolder_path, IMAGE_SIZE, image_list, label_list))
            threads.append(thread)
            thread.start()

    # Attendre que tous les threads soient terminés
    for thread in threads:
        thread.join()
else:
    raise Exception("Dataset folder does not exist")


Dataset folder exists
Reading subfolder: dataset/part001
Reading subfolder: dataset/part003
Reading subfolder: dataset/part004
Reading subfolder: dataset/part009
Reading subfolder: dataset/part012
Reading subfolder: dataset/part000
Reading subfolder: dataset/part011
Reading subfolder: dataset/part007
Reading subfolder: dataset/part002
Reading subfolder: dataset/part006
Reading subfolder: dataset/part008
Reading subfolder: dataset/part013
Reading subfolder: dataset/part010
Reading subfolder: dataset/part014
Reading subfolder: dataset/part005
Finished reading subfolder: dataset/part014
Finished reading subfolder: dataset/part011
Finished reading subfolder: dataset/part003
Finished reading subfolder: dataset/part012
Finished reading subfolder: dataset/part013
Finished reading subfolder: dataset/part007
Finished reading subfolder: dataset/part009
Finished reading subfolder: dataset/part006
Finished reading subfolder: dataset/part002
Finished reading subfolder: dataset/part004
Finished read

In [27]:
colors_lst = [ 
    [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], 
    [255, 0, 255], [0, 255, 255], [128, 0, 0], [0, 128, 0], 
    [0, 0, 128], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
    [192, 0, 0], [0, 192, 0], [0, 0, 192], [192, 192, 0], 
    [192, 0, 192], [0, 192, 192], [64, 0, 0], [0, 64, 0], 
    [0, 0, 64], [64, 64, 0], [64, 0, 64], [0, 64, 64]
]
# Function to apply watershed segmentation
def apply_watershed(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2)
    
    sure_bg = cv2.dilate(opening, kernel, iterations = 3)
    
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0)
    
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    
    markers = cv2.watershed(image, markers)
    
    new_image = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
    for i in range(2, len(colors_lst) + 1):
        new_image[markers == i] = np.array(colors_lst[i - 1]) * 0.2 + image[markers == i] * 0.8
        
    new_image[markers == -1] = image[markers == -1]
    
    return new_image

In [39]:
def apply_kmeans(image, k = 5) :
    Z = image.reshape((-1,3))
    Z = np.float32(Z)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = k
    ret,label,center=cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    
    return res2

In [40]:
# Display the segmented images as a video stream
for img in image_list:
    
    cv2.imshow('Segmented Image', apply_kmeans(img))
    if cv2.waitKey(18) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(18) & 0xFF == ord('s'):
        cv2.imwrite("segmented_image.png", apply_kmeans(img))
    

cv2.destroyAllWindows()